In [42]:
import numpy as np
import CoolProp.CoolProp as CP
import math
import plotly.offline as pyo
import plotly.graph_objects as go

#GEOMETRY FROM NTOPOLOGY
vf1 = 2606
vt = 8000
asf1 = 6686
L=20/1000

epsi = vf1/8000
#porosity
#epsif1 = vf1/vthex


In [50]:

#air props
muair = CP.PropsSI('V', 'T', 300, 'P', 101325, 'Air')  # air dynamic viscosity
rhoair = CP.PropsSI('D', 'T', 300, 'P', 101325, 'Air')  # air density

# water props
muw = CP.PropsSI('V', 'T', 300, 'P', 101325, 'Water')  # water dyn visc
rhow = CP.PropsSI('D', 'T', 300, 'P', 101325, 'Water')  # water density

mteste = np.linspace(0.05, 0.13, 10)

mair = np.linspace(0.1, 5, 10) #kg/s
mw1 = np.linspace(0.01, 0.1, 10) #kg/s
mw2 = np.linspace(0.01, 0.1, 10) #kg/s





def Dh(vf, afs): #OBTIDO NO ARTIGO QUE PROF KLEBER ENVIOU (NÃO AUTORIZADO COMPARTILHAMENTO)
    '''
    vf = representa o volume de fluído
    afs = área de interface solido/fluido ( verificar a definição no artigo nao publicado, ambas equações são iguais)
    '''
    Dh = (4*vf)/afs
    return Dh
#hydraulic diameter Dh
Dhf1 = Dh(vf1, asf1)
print(Dhf1)
print(asf1)


def Re(m,Dh,mu,Across): #reynolds
    Re = (m*Dh)/(Across*mu)
    return Re
#reynolds calculation Re(m,Dh,mu,Across)
Ref1 = Re(mair, Dhf1/1e3, asf1/1e6,muw)
print(Ref1)

V = (muw*Ref1)/(rhow*(Dhf1/1000))



1.5590786718516303
6686
[ 1257.92350058  8106.61811487 14955.31272915 21804.00734343
 28652.70195772 35501.396572   42350.09118628 49198.78580057
 56047.48041485 62896.17502913]


In [51]:

#------------- f CORRELATIONS --------------------#
#F CORRELATION_1
def fcorrelation1(Re): # kaixin Yan et al. 2024 correlation
    f = 2.577*(np.power(Re, -0.095)) 
    return f
f1air = fcorrelation1(Ref1)
#f1w1 = fcorrelation1(Ref2)
#f1w2 = fcorrelation1(Ref3)

#F CORRELATION_2
def fcorrelation2(Re): #kaixin Yan et al #GYROID AS PRINTED
    f = 2.756*(np.power(Re, -0.166))  
    return f
f2air = fcorrelation2(Ref1)
#f2w1 = fcorrelation2(Ref2)
#f2w2 = fcorrelation2(Ref3)

#F CORRELATION_3
def fcorrelation3(Re): # kaixin Yan et al #GYROID AS POLISHED 
    f = 2.344*(np.power(Re, -0.081)) 
    return f
f3air = fcorrelation3(Ref1)
#f3w1 = fcorrelation3(Ref2)
#f3w2 = fcorrelation3(Ref3)



#--------------------PRESSURE DROP------------------------#
def dP(f,L,epsi,Dh,rho,V): #REYNOLDS et al.
    dP = (f*L*2*rho*(np.power(V, 2))) / (np.power(epsi, 2)*Dh) 
    return dP

# pressuredrop for fcorrelation1
Dp_fcor1_f1 = dP(f1air, L, epsi,Dhf1/1000,rhow,V)
#Dp_fcor1_f2 = dP(Lf1f2, epsif2, Dhf2, rhow, Vw1, f1w1)
#Dp_fcor1_f3 = dP(Lf3, epsif3, Dhf3, rhow, Vw2, f1w2)

# pressuredrop for fcorrelation2
#Dp_fcor2_f1 = dP(Lf1f2, epsif1, Dhf1, rhoair, Var, f2air)
#Dp_fcor2_f2 = dP(Lf1f2, epsif2, Dhf2, rhow, Vw1, f2w1)
#Dp_fcor2_f3 = dP(Lf3, epsif3, Dhf3, rhow, Vw2, f2w2)

# pressuredrop for fcorrelation3
#Dp_fcor3_f1 = dP(Lf1f2, epsif1, Dhf1, rhoair, Var, f3air)
#Dp_fcor3_f2 = dP(Lf1f2, epsif2, Dhf2, rhow, Vw1, f3w1)
#Dp_fcor3_f3 = dP(Lf3, epsif3, Dhf3, rhow, Vw2, f3w2)

In [45]:
# PLOT dP/Re for fcorrelation1
trace1 = go.Scatter(x=Ref1, y=Dp_fcor1_f1, mode='markers', name='dP_Air_fcorrelation1', marker=dict(color='blue'))
#trace2 = go.Scatter(x=Ref1, y=Dp_fcor2_f1, mode='markers', name='dP_Air_fcorrelation2', marker=dict(color='red'))
#trace3 = go.Scatter(x=Ref1, y=Dp_fcor3_f1, mode='markers', name='dP_Air_fcorrelation3', marker=dict(color='green'))

layout = go.Layout(title='Dp/Re', xaxis=dict(title='Reynolds Number'), yaxis=dict(title='Pressure Drop (dP)'))

fig = go.Figure(data=[trace1], layout=layout)
pyo.plot(fig)

print(f2air)

[1.59162592 1.16819734 1.05527962 0.9912577  0.94731435 0.91420346
 0.88782176 0.86600256 0.84746797 0.83140376]


In [46]:

# PLOT dP/Re for fcorrelation1
trace1 = go.Scatter(x=Ref2, y=Dp_fcor1_f2, mode='markers', name='dP_WaterF2_fcorrelation1', marker=dict(color='blue'))
trace2 = go.Scatter(x=Ref2, y=Dp_fcor2_f2, mode='markers', name='dP_WaterF2_fcorrelation2', marker=dict(color='red'))
trace3 = go.Scatter(x=Ref2, y=Dp_fcor3_f2, mode='markers', name='dP_WaterF2_fcorrelation3', marker=dict(color='green'))

layout = go.Layout(title='Dp/Re', xaxis=dict(title='Reynolds Number'), yaxis=dict(title='Pressure Drop (dP)'))

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
pyo.plot(fig)

print(f1air)

NameError: name 'Ref2' is not defined

In [ ]:
# PLOT dP/Re for fcorrelation1
trace1 = go.Scatter(x=Ref3, y=Dp_fcor1_f3, mode='markers', name='dP_WaterF3_fcorrelation1', marker=dict(color='blue'))
trace2 = go.Scatter(x=Ref3, y=Dp_fcor2_f3, mode='markers', name='dP_WaterF3_fcorrelation2', marker=dict(color='red'))
trace3 = go.Scatter(x=Ref3, y=Dp_fcor3_f3, mode='markers', name='dP_WaterF3_fcorrelation3', marker=dict(color='green'))

layout = go.Layout(title='Dp/Re', xaxis=dict(title='Reynolds Number'), yaxis=dict(title='Pressure Drop (dP)'))

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
pyo.plot(fig)

print(f1air)

[3.93941914 3.90018519 3.86503049 3.83321352 3.80417608 3.77748776
 3.75280985 3.72987117 3.70845131 3.68836881]
